## Measure the similarity between images by calculating their histograms

A histogram is a very important tool in Image processing. It is a graphical representation of the distribution of data. An image histogram gives a graphical representation of the distribution of pixel intensities in a digital image.

The x-axis indicates the range of values the variable can take. This range can be divided into a series of intervals called bins. 

The y-axis shows the count of how many values fall within that interval or bin.
When plotting the histogram we have the pixel intensity in the X-axis and the frequency in the Y-axis. As any other histogram we can decide how many bins to use.

A histogram can be calculated both for the gray-scale image and for the colored image.
In the first case we have a single channel, hence a single histogram. In the second case we have 3 channels, hence 3 histograms.

Calculating the histogram of an image is very useful as it gives an intuition regarding some properties of the image such as the tonal range, the contrast and the brightness.

* To identify the dominant colors in an image, we can use the histogram plot of the Hue channel.